<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Fetch HTML content
url = "https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue"
response = requests.get(url)
html_content = response.content

# Parse HTML content
soup = BeautifulSoup(html_content, 'html.parser')

In [3]:
tables = soup.find_all('table')
for i, table in enumerate(tables):
    print(f"Table {i} Class: {table.get('class')}")

Table 0 Class: ['wikitable', 'sortable', 'sticky-header-multi', 'sort-under']
Table 1 Class: ['wikitable', 'sortable', 'plainrowheaders']
Table 2 Class: ['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
Table 3 Class: ['nowraplinks', 'navbox-subgroup']
Table 4 Class: ['nowraplinks', 'navbox-subgroup']
Table 5 Class: ['nowraplinks', 'navbox-subgroup']


In [4]:
# Assuming table index 0 based on your HTML
table = tables[0]  # Adjust the index if needed

# Check if table is found correctly
if table:
    print("Table found and selected.")
else:
    print("Table not found.")

Table found and selected.


In [10]:
# Extract headers
headers = []
header_rows = table.find_all('tr')

for header in header_rows[0].find_all('th'):
    sup_tag = header.find('sup', {'class': 'reference'})
    if sup_tag and 'note 1' in sup_tag.text:
        sup_tag.extract()
    headers.append(header.text.strip())

# Second row headers (if applicable)
if len(header_rows) > 1:
    for header in header_rows[1].find_all('th'):
        headers.append(header.text.strip())

# Extract rows
rows = []
for row in table.find_all('tr')[2:]:  # Skip header rows
    cells = row.find_all(['td', 'th'])
    cell_texts = []
    for cell in cells:
        cell_text = cell.text.strip()
        # Check for images representing 'No' and 'Yes'
        if cell.find('img', {'alt': 'No'}):
            cell_text = 'No'
        elif cell.find('img', {'alt': 'Yes'}):
            cell_text = 'Yes'
        cell_texts.append(cell_text)
    rows.append(cell_texts)

# Ensure the rows have the same number of columns as headers
for i in range(len(rows)):
    while len(rows[i]) < len(headers):
        rows[i].append('')  # Fill missing cells with empty string


In [12]:
# Convert to DataFrame
df = pd.DataFrame(rows, columns=headers)

# Display the DataFrame
df.head()

,Rank,Ram,Industry,Revenue,Profit,Employees,Headquarters,State-owned,Ref.,USD millions
0,1,Walmart,Retail,"$648,125","$15,511","2,100,000",United States,No,[1],
1,2,Amazon,Retail information technology,"$574,785","$30,425","1,525,000",United States,No,[4],
2,3,State Grid Corporation of China,Electricity,"$545,948","$9,204","1,361,423",China,Yes,[5],
3,4,Saudi Aramco,Oil and gas,"$488,980","$129,699","73,311",Saudi Arabia,Yes,[6],
4,5,China Petrochemical Corporation,Oil and gas,"$429,700","$9,393","513,434",China,Yes,[7],


In [7]:
# Drop the unnecessary column "USD millions" 
df = df.drop(columns=['USD millions']) 

In [9]:
# Display the DataFrame without index numbers using the style attribute 
df.style.hide(axis="index")

Rank,Ram,Industry,Revenue,Profit,Employees,Headquarters[note 1],State-owned,Ref.
1,Walmart,Retail,"$648,125","$15,511","2,100,000",United States,No,[1]
2,Amazon,Retail information technology,"$574,785","$30,425","1,525,000",United States,No,[4]
3,State Grid Corporation of China,Electricity,"$545,948","$9,204","1,361,423",China,Yes,[5]
4,Saudi Aramco,Oil and gas,"$494,890","$129,699","73,311",Saudi Arabia,Yes,[6]
5,China Petrochemical Corporation,Oil and gas,"$429,700","$9,393","513,434",China,Yes,[7]
6,China National Petroleum Corporation,Oil and gas,"$476,000","$25,250","1,026,301",China,Yes,[8]
7,Vitol,Commodities,"$400,000","$13,000","1,560",Switzerland,No,[9][10]
8,Apple,Information technology,"$383,285","$96,995","161,000",United States,No,[11]
9,UnitedHealth Group,Healthcare,"$371,622","$22,381","440,000",United States,No,[12]
10,Berkshire Hathaway,Financials,"$364,482","$96,223","396,500",United States,No,[13]


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Rank                  50 non-null     object
 1   Ram                   50 non-null     object
 2   Industry              50 non-null     object
 3   Revenue               50 non-null     object
 4   Profit                50 non-null     object
 5   Employees             50 non-null     object
 6   Headquarters[note 1]  50 non-null     object
 7   State-owned           50 non-null     object
 8   Ref.                  50 non-null     object
dtypes: object(9)
memory usage: 3.6+ KB


In [10]:
# Save the DataFrame to a CSV file without index numbers 
df.to_csv('largest_companies_by_revenue.csv', index=False)

In [34]:
print(df['State-owned'].unique())

['No' 'Yes']
